<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL-Tutorials/blob/master/1-Introduction/Stock_NeurIPS2018_SB3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Reinforcement Learning for Stock Trading from Scratch: Multiple Stock Trading

* **Pytorch Version** 



# Content

* [1. Task Description](#0)
* [2. Install Python packages](#1)
    * [2.1. Install Packages](#1.1)    
    * [2.2. A List of Python Packages](#1.2)
    * [2.3. Import Packages](#1.3)
    * [2.4. Create Folders](#1.4)
* [3. Download and Preprocess Data](#2)
* [4. Preprocess Data](#3)        
    * [4.1. Technical Indicators](#3.1)
    * [4.2. Perform Feature Engineering](#3.2)
* [5. Build Market Environment in OpenAI Gym-style](#4)  
    * [5.1. Data Split](#4.1)  
    * [5.3. Environment for Training](#4.2)    
* [6. Train DRL Agents](#5)
* [7. Backtesting Performance](#6)  
    * [7.1. BackTestStats](#6.1)
    * [7.2. BackTestPlot](#6.2)   
  

<a id='0'></a>
# Part 1. Task Discription

We train a DRL agent for stock trading. This task is modeled as a Markov Decision Process (MDP), and the objective function is maximizing (expected) cumulative return.

We specify the state-action-reward as follows:

* **State s**: The state space represents an agent's perception of the market environment. Just like a human trader analyzing various information, here our agent passively observes many features and learns by interacting with the market environment (usually by replaying historical data).

* **Action a**: The action space includes allowed actions that an agent can take at each state. For example, a ∈ {−1, 0, 1}, where −1, 0, 1 represent
selling, holding, and buying. When an action operates multiple shares, a ∈{−k, ..., −1, 0, 1, ..., k}, e.g.. "Buy
10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or −10, respectively

* **Reward function r(s, a, s′)**: Reward is an incentive for an agent to learn a better policy. For example, it can be the change of the portfolio value when taking a at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio values at state s′ and s, respectively


**Market environment**: 30 consituent stocks of Dow Jones Industrial Average (DJIA) index. Accessed at the starting date of the testing period.


The data for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Install Python Packages

<a id='1.1'></a>
## 2.1. Install packages


In [1]:
## install finrl library
!pip install wrds
!pip install swig
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to /tmp/pip-req-build-ovpsxg7d
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git /tmp/pip-req-build-ovpsxg7d
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit d22c4a53c20e3f8d159712a3772ac565afaf3176
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-js9vl1cf/elegantrl_51e34ae9df0a4b3cbadd982024a7ecdd
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-js9vl1cf/elegantrl_51e34ae9df0a4b3cbadd982024a7ecdd
  Resolved https://github.com/AI4Finance-Foundation/ElegantRL.git to commit dde7ee4daa226450a43280e4ee02f1f23b565aa2
  Preparing metadata (setup.py) ... done



<a id='1.2'></a>
## 2.2. A list of Python packages 
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools

/workspaces/rl-baseline/.venv/lib/python3.10/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


<a id='1.4'></a>
## 2.4. Create Folders

In [3]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])



<a id='2'></a>
# Part 3. Download Data
Yahoo Finance provides stock data, financial news, financial reports, etc. Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** in FinRL-Meta to fetch data via Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).



-----
class YahooDownloader:
    Retrieving daily stock data from
    Yahoo Finance API

    Attributes
    ----------
        start_date : str
            start date of the data (modified from config.py)
        end_date : str
            end date of the data (modified from config.py)
        ticker_list : list
            a list of stock tickers (modified from config.py)

    Methods
    -------
    fetch_data()


In [4]:
# from config.py, TRAIN_START_DATE is a string
print(TRAIN_START_DATE)
# from config.py, TRAIN_END_DATE is a string
print(TRAIN_END_DATE)

2014-01-06
2020-07-31


In [5]:
TRAIN_START_DATE = '2003-01-01'
TRAIN_END_DATE = '2020-07-01'
TRADE_START_DATE = '2020-07-01'
TRADE_END_DATE = '2021-10-31'


In [6]:
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = ['MSFT', 'TSLA', 'AMZN', 'NFLX']).fetch_data()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
Shape of DataFrame:  (17079, 8)


In [7]:
print(df.head(5))

         date       open     high        low      close     volume   tic  day
0  2003-01-02   0.959500   0.9840   0.927500   0.978500  129680000  AMZN    3
1  2003-01-02  26.150000  26.8750  25.855000  16.686026   67025200  MSFT    3
2  2003-01-02   0.846429   0.9000   0.810714   0.817857    8397200  NFLX    3
3  2003-01-03   0.979500   1.0265   0.970500   1.026000  175140000  AMZN    4
4  2003-01-03  26.795000  26.9000  26.440001  16.707771   55546800  MSFT    4


In [8]:
def subset_df(input_df, ticker):
    sub_df = input_df[df['tic'] == ticker]
    return sub_df

In [9]:
MSFT_df = subset_df(df, ticker = 'MSFT')
print(MSFT_df.head(5))
print(MSFT_df.shape)

          date       open       high        low      close    volume   tic  \
1   2003-01-02  26.150000  26.875000  25.855000  16.686026  67025200  MSFT   
4   2003-01-03  26.795000  26.900000  26.440001  16.707771  55546800  MSFT   
7   2003-01-06  27.010000  27.615000  26.900000  17.012177  59456200  MSFT   
10  2003-01-07  27.459999  28.004999  27.340000  17.332088  80517400  MSFT   
13  2003-01-08  27.684999  27.775000  27.055000  16.847544  64566000  MSFT   

    day  
1     3  
4     4  
7     0  
10    1  
13    2  
(4741, 8)


In [10]:
TSLA_df = subset_df(df, ticker = 'TSLA')
print(TSLA_df.head(5))
print(TSLA_df.shape)

            date      open      high       low     close     volume   tic  day
5658  2010-06-29  1.266667  1.666667  1.169333  1.592667  281494500  TSLA    1
5662  2010-06-30  1.719333  2.028000  1.553333  1.588667  257806500  TSLA    2
5666  2010-07-01  1.666667  1.728000  1.351333  1.464000  123282000  TSLA    3
5670  2010-07-02  1.533333  1.540000  1.247333  1.280000   77097000  TSLA    4
5674  2010-07-06  1.333333  1.333333  1.055333  1.074000  103003500  TSLA    1
(2856, 8)


In [11]:
AMZN_df = subset_df(df, ticker = 'AMZN')
print(AMZN_df.head(5))
print(AMZN_df.shape)

          date    open    high     low   close     volume   tic  day
0   2003-01-02  0.9595  0.9840  0.9275  0.9785  129680000  AMZN    3
3   2003-01-03  0.9795  1.0265  0.9705  1.0260  175140000  AMZN    4
6   2003-01-06  1.0200  1.0585  1.0150  1.0350  153962000  AMZN    0
9   2003-01-07  1.0415  1.0875  1.0300  1.0775  191364000  AMZN    1
12  2003-01-08  1.0720  1.0800  1.0410  1.0510  139630000  AMZN    2
(4741, 8)


In [12]:
NFLX_df = subset_df(df, ticker = 'NFLX')
print(NFLX_df.head(5))
print(NFLX_df.shape)

          date      open      high       low     close   volume   tic  day
2   2003-01-02  0.846429  0.900000  0.810714  0.817857  8397200  NFLX    3
5   2003-01-03  0.850000  0.850000  0.771429  0.772857  3248000  NFLX    4
8   2003-01-06  0.785714  0.800000  0.762143  0.792857  3003000  NFLX    0
11  2003-01-07  0.771429  0.795714  0.771429  0.783571  1937600  NFLX    1
14  2003-01-08  0.771429  0.857143  0.762143  0.815000  5356400  NFLX    2
(4741, 8)


In [13]:
# 5+

In [14]:
# df = YahooDownloader(start_date = TRAIN_START_DATE,
#                      end_date = TRADE_END_DATE,
#                      ticker_list = config_tickers.DOW_30_TICKER).fetch_data()

In [15]:
# print(config_tickers.DOW_30_TICKER)

In [16]:
# df.shape

In [17]:
# df.sort_values(['date','tic'],ignore_index=True).head()

# Part 4: Preprocess Data
We need to check for missing data and do feature engineering to convert the data point into a state.
* **Adding technical indicators**. In practical trading, various information needs to be taken into account, such as historical prices, current holding shares, technical indicators, etc. Here, we demonstrate two trend-following technical indicators: MACD and RSI.
* **Adding turbulence index**. Risk-aversion reflects whether an investor prefers to protect the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the turbulence index that measures extreme fluctuation of asset price.

In [18]:
MSFT_fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

MSFT_processed = MSFT_fe.preprocess_data(MSFT_df)

Successfully added technical indicators
[*********************100%%**********************]  1 of 1 completed
Shape of DataFrame:  (4740, 8)
Successfully added vix
Successfully added turbulence index


In [19]:
list_ticker = MSFT_processed["tic"].unique().tolist()
list_date = list(pd.date_range(MSFT_processed['date'].min(), MSFT_processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

MSFT_processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(MSFT_processed ,on=["date","tic"],how="left")
MSFT_processed_full = MSFT_processed_full[MSFT_processed_full['date'].isin(MSFT_processed['date'])]
MSFT_processed_full = MSFT_processed_full.sort_values(['date','tic'])

MSFT_processed_full = MSFT_processed_full.fillna(0)

In [20]:
MSFT_processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2003-01-02,MSFT,26.150000,26.875000,25.855000,16.686026,67025200.0,3.0,0.000000,16.727651,16.666145,100.000000,66.666667,100.000000,16.686026,16.686026,25.389999,0.0
1,2003-01-03,MSFT,26.795000,26.900000,26.440001,16.707771,55546800.0,4.0,0.000488,16.727651,16.666145,100.000000,66.666667,100.000000,16.696898,16.696898,24.680000,0.0
2,2003-01-06,MSFT,27.010000,27.615000,26.900000,17.012177,59456200.0,0.0,0.010061,17.166692,16.437291,100.000000,100.000000,100.000000,16.801991,16.801991,24.910000,0.0
3,2003-01-07,MSFT,27.459999,28.004999,27.340000,17.332088,80517400.0,1.0,0.025719,17.542524,16.326507,100.000000,99.009805,100.000000,16.934515,16.934515,25.129999,0.0
4,2003-01-08,MSFT,27.684999,27.775000,27.055000,16.847544,64566000.0,2.0,0.014405,17.449387,16.384855,55.865640,37.915833,56.019308,16.917121,16.917121,25.530001,0.0
5,2003-01-09,MSFT,27.360001,27.959999,27.264999,17.335205,61808000.0,3.0,0.028185,17.572613,16.400990,69.760609,78.359119,61.841807,16.986802,16.986802,24.250000,0.0
6,2003-01-10,MSFT,27.549999,28.150000,27.450001,17.369371,67730200.0,4.0,0.038085,17.649413,16.433496,70.435228,90.017376,66.684060,17.041455,17.041455,24.320000,0.0
7,2003-01-13,MSFT,28.260000,28.375000,27.885000,17.515358,60817800.0,0.0,0.051054,17.755752,16.445633,73.088942,111.324561,71.295181,17.100693,17.100693,24.900000,0.0
8,2003-01-14,MSFT,28.165001,28.500000,28.094999,17.695509,54206400.0,1.0,0.068196,17.896655,16.436911,75.855528,114.054869,73.475371,17.166783,17.166783,24.570000,0.0
9,2003-01-15,MSFT,28.500000,28.660000,28.094999,17.478077,59622600.0,2.0,0.068164,17.913653,16.482172,67.226461,96.734103,75.972398,17.197913,17.197913,25.510000,0.0


In [21]:
# 5+

SyntaxError: invalid syntax (4281351646.py, line 1)

<a id='4'></a>
# Part 5. Build A Market Environment in OpenAI Gym-style
The training process involves observing stock price change, taking an action and reward's calculation. By interacting with the market environment, the agent will eventually derive a trading strategy that may maximize (expected) rewards.

Our market environment, based on OpenAI Gym, simulates stock markets with historical market data.

## Data Split
We split the data into training set and testing set as follows:

Training data period: 2009-01-01 to 2020-07-01

Trading data period: 2020-07-01 to 2021-10-31


In [22]:
MSFT_train = data_split(MSFT_processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
MSFT_trade = data_split(MSFT_processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(MSFT_train))
print(len(MSFT_trade))

4404
336


In [23]:
MSFT_train.tail()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
4399,2020-06-24,MSFT,201.600006,203.250000,196.559998,192.088364,36740600.0,2.0,4.715499,196.994507,172.484309,58.946928,146.456068,26.607861,182.238207,174.415711,33.840000,0.755765
4400,2020-06-25,MSFT,197.800003,200.610001,195.470001,194.515717,27803900.0,3.0,4.814099,197.952417,173.365340,60.055350,127.469897,22.709235,182.920745,175.112644,32.220001,0.178882
4401,2020-06-26,MSFT,199.729996,199.889999,194.880005,190.622253,54675800.0,4.0,4.525898,198.211058,174.376671,57.480191,97.130778,20.561175,183.448328,175.835054,34.730000,0.754050
4402,2020-06-29,MSFT,195.779999,198.529999,193.550003,192.670929,26701600.0,0.0,4.411950,198.538060,175.565288,58.449995,86.958207,15.681406,183.959318,176.540777,31.780001,0.122255
4403,2020-06-30,MSFT,197.880005,204.399994,197.740005,197.593506,34310300.0,1.0,4.665079,199.786912,176.122360,60.679236,130.581469,29.201117,184.577913,177.351618,30.430000,0.867399


In [24]:
MSFT_trade.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2020-07-01,MSFT,203.139999,206.350006,201.770004,198.748917,32061200.0,2.0,4.902407,201.034482,176.752563,61.184891,140.812367,33.015146,185.276334,177.997107,28.620001,0.023185
1,2020-07-02,MSFT,205.679993,208.020004,205.000000,200.263550,29315800.0,3.0,5.153304,201.958580,178.094609,61.850192,146.838407,36.163399,185.943036,178.696564,27.680000,0.049385
2,2020-07-06,MSFT,208.830002,211.130005,208.089996,204.574493,31897600.0,0.0,5.635042,204.092355,178.242516,63.682996,163.572060,41.625094,186.825610,179.441406,27.940001,0.592298
3,2020-07-07,MSFT,210.449997,214.669998,207.990005,202.195709,33600700.0,1.0,5.758495,205.334808,178.931238,61.983174,153.212952,47.129113,187.626302,180.146439,29.430000,0.290708
4,2020-07-08,MSFT,210.070007,213.259995,208.690002,206.642563,33600000.0,2.0,6.144328,207.406231,179.095862,63.849205,152.108551,47.129113,188.638007,180.919616,28.080000,0.615544


In [25]:
INDICATORS

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [27]:
stock_dimension = len(MSFT_train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


Stock Dimension: 1, State Space: 11


In [ ]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)


## Environment for Training



In [ ]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<a id='5'></a>
# Part 6: Train DRL Agents
* The DRL algorithms are from **Stable Baselines 3**. Users are also encouraged to try **ElegantRL** and **Ray RLlib**.
* FinRL includes fine-tuned standard DRL algorithms, such as DQN, DDPG, Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

In [ ]:
agent = DRLAgent(env = env_train)

if_using_a2c = False
if_using_ddpg = False
if_using_ppo = False
if_using_td3 = False
if_using_sac = True


### Agent Training: 5 algorithms (A2C, DDPG, PPO, TD3, SAC)


### Agent 1: A2C


In [ ]:
agent = DRLAgent(env = env_train)
model_a2c = agent.get_model("a2c")

if if_using_a2c:
  # set up logger
  tmp_path = RESULTS_DIR + '/a2c'
  new_logger_a2c = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_a2c.set_logger(new_logger_a2c)


In [ ]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=50000) if if_using_a2c else None

### Agent 2: DDPG

In [ ]:
agent = DRLAgent(env = env_train)
model_ddpg = agent.get_model("ddpg")

if if_using_ddpg:
  # set up logger
  tmp_path = RESULTS_DIR + '/ddpg'
  new_logger_ddpg = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ddpg.set_logger(new_logger_ddpg)

In [ ]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=50000) if if_using_ddpg else None

### Agent 3: PPO

In [ ]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

if if_using_ppo:
  # set up logger
  tmp_path = RESULTS_DIR + '/ppo'
  new_logger_ppo = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ppo.set_logger(new_logger_ppo)

In [ ]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=50000) if if_using_ppo else None

### Agent 4: TD3

In [ ]:
agent = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 100, 
              "buffer_size": 1000000, 
              "learning_rate": 0.001}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

if if_using_td3:
  # set up logger
  tmp_path = RESULTS_DIR + '/td3'
  new_logger_td3 = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_td3.set_logger(new_logger_td3)

In [ ]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=30000) if if_using_td3 else None

### Agent 5: SAC

In [ ]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 100000,
    "learning_rate": 0.0001,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

if if_using_sac:
  # set up logger
  tmp_path = RESULTS_DIR + '/sac'
  new_logger_sac = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_sac.set_logger(new_logger_sac)

In [ ]:

trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps = 40000) if if_using_sac else None

## In-sample Performance

Assume that the initial capital is $1,000,000.

### Set turbulence threshold
Set the turbulence threshold to be greater than the maximum of insample turbulence data. If current turbulence index is greater than the threshold, then we assume that the current market is volatile

In [ ]:
data_risk_indicator = processed_full[(processed_full.date<TRAIN_END_DATE) & (processed_full.date>=TRAIN_START_DATE)]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

In [ ]:
insample_risk_indicator.vix.describe()

In [ ]:
insample_risk_indicator.vix.quantile(0.996)

In [ ]:
insample_risk_indicator.turbulence.describe()

In [ ]:
insample_risk_indicator.turbulence.quantile(0.996)

### Trading (Out-of-sample Performance)

We update periodically in order to take full advantage of the data, e.g., retrain quarterly, monthly or weekly. We also tune the parameters along the way, in this notebook we use the in-sample data from 2009-01 to 2020-07 to tune the parameters once, so there is some alpha decay here as the length of trade date extends. 

Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [ ]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70,risk_indicator_col='vix', **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

In [ ]:
trade.head()

In [ ]:
trained_moedl = trained_sac
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_moedl, 
    environment = e_trade_gym)

In [ ]:
df_account_value.shape

In [ ]:
df_account_value.tail()

In [ ]:
df_actions.head()

<a id='6'></a>
# Part 7: Backtesting Results
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [ ]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

In [ ]:
#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^DJI", 
        start = df_account_value.loc[0,'date'],
        end = df_account_value.loc[len(df_account_value)-1,'date'])

stats = backtest_stats(baseline_df, value_col_name = 'close')


In [ ]:
df_account_value.loc[0,'date']

In [ ]:
df_account_value.loc[len(df_account_value)-1,'date']

<a id='6.2'></a>
## 7.2 BackTestPlot

In [ ]:
print("==============Compare to DJIA===========")
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
backtest_plot(df_account_value, 
             baseline_ticker = '^DJI', 
             baseline_start = df_account_value.loc[0,'date'],
             baseline_end = df_account_value.loc[len(df_account_value)-1,'date'])